### Human in the Loop and Multiple Termination Condition

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console

from dotenv import load_dotenv

import os

In [ ]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")
azure_openai_deployment_name = os.getenv(
    "AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo")

if not azure_openai_api_key or \
        not azure_openai_endpoint or \
        not azure_openai_api_version or \
        not azure_openai_deployment_name:
    raise ValueError(
        "Please set Azure Open AI Endpoint Details in your environment variables.")

In [ ]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [ ]:
assistant = AssistantAgent(
    name="Assistant",
    model_client=model_client,
    system_message="You are a helpful assistant.",
)

user_proxy = UserProxyAgent(
    name="UserProxy",
    input_func=input,
)

text_termination = TextMentionTermination("APPROVE")
max_messages_termination = MaxMessageTermination(10)
combined_termination = text_termination | max_messages_termination

team = RoundRobinGroupChat([assistant, user_proxy],
                           termination_condition=combined_termination,
                           )

In [ ]:
await Console(
    team.run_stream(
        task="Suggest three creative ideams for a new mobile app."))